In [2]:
from tqdm import tqdm
# from datasets import load_dataset
import re
import json, sys, os, glob
import numpy as np
from typing import List, Tuple
import math
import matplotlib.pyplot as plt
import re
import re
import ast
import tiktoken

def validate_operations(initial_numbers, operations, target):
    """
    Validates if the operations use all initial numbers exactly once and reach the target.
    """
    # Make a copy of initial numbers to track usage
    available_numbers = initial_numbers.copy()
    # Dictionary to store intermediate results
    intermediate_results = {}
    
    for i, operation in enumerate(operations):
        # Parse the operation (format like '9-5=4')
        parts = operation.split('=')
        if len(parts) != 2:
            return False
        
        result_str = parts[1].strip()
        expression = parts[0].strip()
        
        # Find operator (+, -, *, /)
        op_match = re.search(r'[+\-*/]', expression)
        if not op_match:
            return False
        
        operator = op_match.group(0)
        operands = expression.split(operator)
        if len(operands) != 2:
            return False
        
        left_operand = operands[0].strip()
        right_operand = operands[1].strip()
        
        # Try to get values for operands
        left_val = None
        if left_operand.isdigit() or (left_operand.startswith('-') and left_operand[1:].isdigit()):
            left_val = int(left_operand)
            if left_val in available_numbers:
                available_numbers.remove(left_val)
            elif left_val not in intermediate_results.values():
                return False  # Not an available number or intermediate result
        else:
            return False  # Non-numeric operand
        
        right_val = None
        if right_operand.isdigit() or (right_operand.startswith('-') and right_operand[1:].isdigit()):
            right_val = int(right_operand)
            if right_val in available_numbers:
                available_numbers.remove(right_val)
            elif right_val not in intermediate_results.values():
                return False  # Not an available number or intermediate result
        else:
            return False  # Non-numeric operand
        
        # Calculate the result
        calculated_result = None
        if operator == '+':
            calculated_result = left_val + right_val
        elif operator == '-':
            calculated_result = left_val - right_val
        elif operator == '*':
            calculated_result = left_val * right_val
        elif operator == '/':
            if right_val == 0:
                return False  # Division by zero
            calculated_result = left_val / right_val
        
        # Verify the result matches what's stated
        result_val = int(result_str) if result_str.isdigit() else None
        if result_val is None or abs(calculated_result - result_val) > 1e-10:
            return False
        
        # Store intermediate result for future operations
        intermediate_results[f"step{i}"] = result_val
    
    # Check if all initial numbers were used and final result matches target
    return len(available_numbers) == 0 and result_val == target


def evaluate_countdown_trajectory_claude(ds_entry) -> Tuple[bool, str]:
    """
    Given:
      - target: the desired final result.
      - nums: list of initial numbers.
      - trajectory: a string that should include lines of the form:
            SOLUTION: YES/NO
            OPERATIONS: [list of strings like 'A+B=C', ...]
            RESULT: final_value
    This function uses regex to extract the parts and then simulates the operations
    by “consuming” initial numbers (and intermediate results) from a pool, verifying:
      • each operation string has one binary operator
      • each operation is valid (left op right equals given result)
      • the operations use all initial numbers exactly once (by simulating removal from pool).
    Returns a tuple (is_valid, message) where is_valid is True only if the trajectory is correct.
    """
    # target: int, nums: List[int], trajectory: str
    trajectory = ds_entry['completion']
    target = ds_entry['target']
    nums = ds_entry['nums']
    # Extract SOLUTION (YES or NO)
    sol_match = re.search(r"SOLUTION:\s*(YES|NO)", trajectory, re.IGNORECASE)
    if not sol_match:
        return False, "Could not find SOLUTION declaration."
    sol_decl = sol_match.group(1).upper()
    if sol_decl != "YES":
        return False, "The trajectory indicates no valid solution."

    # Extract OPERATIONS list
    ops_match = re.search(r"OPERATIONS:\s*(\[[^\]]*\])", trajectory, re.DOTALL)
    if not ops_match:
        return False, "Could not find OPERATIONS list."
    try:
        operations = ast.literal_eval(ops_match.group(1))
        if not isinstance(operations, list):
            return False, "OPERATIONS is not a list."
    except Exception as e:
        return False, f"Failed to parse OPERATIONS list: {e}"
        
    # Extract final RESULT
    res_match = re.search(r"RESULT:\s*([-+.\d]+)", trajectory)
    if not res_match:
        return False, "Could not find RESULT."
    try:
        expected_final = float(res_match.group(1))
    except Exception as e:
        return False, f"Failed to parse RESULT: {e}"
    
    # Simulation: available numbers (as floats).
    available = [float(n) for n in nums]
    # We simulate the sequence of operations.
    # Each operation must be of the form "operand1 operator operand2 = result"
    op_pattern = re.compile(r"^\s*([\d.]+)\s*([\+\-\*/])\s*([\d.]+)\s*=\s*([\d.]+)\s*$")
    
    for idx, op_str in enumerate(operations):
        m = op_pattern.match(op_str)
        if not m:
            return False, f"Operation '{op_str}' does not match required pattern."
        op1_str, operator, op2_str, given_result_str = m.groups()
        try:
            op1 = float(op1_str)
            op2 = float(op2_str)
            op_result = float(given_result_str)
        except Exception as e:
            return False, f"Error converting numbers in op '{op_str}': {e}"
            
        # Check that the operation is valid:
        if operator == '+':
            computed = op1 + op2
        elif operator == '-':
            computed = op1 - op2
        elif operator == '*':
            computed = op1 * op2
        elif operator == '/':
            # Avoid division by zero
            if math.isclose(op2,0.0):
                return False, f"Division by zero in op '{op_str}'."
            computed = op1 / op2
        else:
            return False, f"Unknown operator '{operator}' in op '{op_str}'."
            
        if not math.isclose(computed, op_result, rel_tol=1e-5):
            return False, f"In op '{op_str}', computed {computed} which does not match given {op_result}."
            
        # Now simulate consumption:
        # For each operand, check if it is "available". If yes, remove one instance.
        # (We assume that if the operand equals a value in available within tolerance,
        #  it comes from the pool of that operand.)
        def consume(value: float, pool: List[float]) -> bool:
            for i, num in enumerate(pool):
                if math.isclose(num, value, rel_tol=1e-5):
                    del pool[i]
                    return True
            return False
        
        # Try to consume op1 and op2
        if not consume(op1, available):
            return False, f"Operand {op1} in op '{op_str}' not available from initial/intermediate numbers."
        if not consume(op2, available):
            return False, f"Operand {op2} in op '{op_str}' not available from initial/intermediate numbers."
        # Append current operation result to available pool
        available.append(op_result)
    
    # At the end, exactly one number should remain; it should equal the target.
    if len(available) != 1:
        return False, f"After all operations, expected one value but got {len(available)} values: {available}"
    if not math.isclose(available[0], float(target), rel_tol=1e-5):
        return False, f"Final value {available[0]} does not equal the target {target}."
    
    return True, f"Trajectory is valid with operations: {operations}"
    
#     return {
#         'solved': solved,
#         'target': target,
#         'initial_numbers': initial_numbers,
#         'operations': operations,
#         'final_value': final_value
#     }

def evaluate_countdown_trajectory(ds_entry):
    solved, remarks = evaluate_countdown_trajectory_claude(ds_entry)
    enc = tiktoken.get_encoding("cl100k_base")
    tokens = enc.encode(ds_entry['completion'])

    return {
        'solved': solved,
        'target': ds_entry['target'],
        'initial_numbers': ds_entry['nums'],
        'remarks': remarks,
        'completion_length': len(tokens),
    }
        
def evaluate_countdown_trajectories(results_all_trials):
    """
    Evaluates a dataset of countdown problem solver trajectories and returns a flattened
    list of results in the format expected by visualization tools and wandb logging.
    """
    # Create a dict to track which questions were solved in any trial
    question_solved = {i: False for i in range(len(results_all_trials[0]))}
    trial_success_rates = []
    
    # Process each trial
    for i in range(len(results_all_trials)):
        successes_in_trial = 0
        for j in range(len(results_all_trials[i])):
            # problem_text = results_all_trials[i][j].get("prompt", "")
            # solution_text = results_all_trials[i][j].get("completion", "")
            results_all_trials[i][j]['parsed_results'] = evaluate_countdown_trajectory(results_all_trials[i][j])
            
            # Update success counts
            if results_all_trials[i][j]['parsed_results']['solved']:
                successes_in_trial += 1
                question_solved[j] = True
                
        # Calculate success rate for this trial
        trial_success_rates.append(successes_in_trial / len(results_all_trials[i]))
    
    # Calculate aggregate statistics
    best_of_n_successes = sum(question_solved.values())
    best_of_n_rate = best_of_n_successes / len(question_solved)
    mean_of_n_trials = sum(trial_success_rates) / len(trial_success_rates)
    
    # Print summary statistics
    print(f"Success rate for each trial: {trial_success_rates}")
    print(f"\nSummary:")
    print(f"  Best-of-{len(trial_success_rates)} success rate: {best_of_n_rate:.4f} ({best_of_n_successes}/{len(question_solved)})")
    print(f"  Mean success rate across trials: {mean_of_n_trials:.4f}\n")
    
    # Create the properly formatted results array
    # Start with the metrics summary (which will be at index 1 after hyperparams are added)
    final_results = [{
        'trial_success_rates': trial_success_rates,
        'best_of_n': best_of_n_rate,
        'mean': mean_of_n_trials
    }]
    
    # Add individual trajectory results (using best results from any trial)
    for j in range(len(results_all_trials[0])):
        # Find the best result for this problem across all trials
        best_result = None
        for i in range(len(results_all_trials)):
            result = results_all_trials[i][j]
            if best_result is None or (result['parsed_results']['solved'] and not best_result['parsed_results']['solved']):
                best_result = result
        
        # Add this result to our final list
        if best_result:
            final_results.append(best_result)
    
    return final_results


In [26]:
from datasets import load_dataset
import json
# Load the dataset
print("Loading dataset from Hugging Face...")
data = load_dataset("MelinaLaimon/stream-of-search", split="train[0%:80%]")

# load res/output.jsonl
import sys, os, json

os.listdir("./")

with open("../res/output.jsonl", "r") as f:
    lines = f.readlines()
    lines = [json.loads(line) for line in lines]
    
"""
{'id': '74b966fb-9632-472e-8e76-07b5932e7ffd',
 'original_idx': 14,
 'messages': [{'content': "Combine these initial numbers [12, 91, 40, 13] using only arithmetic operations (+, -, *, /) to reach the target value 45. All initial numbers must be used exactly once.\nConclude with the final result in EXACTLY this format:\n```\nSOLUTION: YES/NO\nOPERATIONS: list of string of operations performed, each string involving only 1 operation. For example, ['A+B=C','C+D=E'] is allowed, ['A+B+D=E'] is not allowed\nRESULT: final_value\n```\n\nNote that the solution does exist. Verify your solutions before your present your final results and backtrack to correct mistakes from before your mistakes if you have to.",
   'role': 'user'},
  {'role': 'assistant',
   'content': "```\nSOLUTION: YES\nOPERATIONS: ['91/13=7', '40-7=33', '33+12=45']\nRESULT: 45\n```",
   'reasoning_content': "Okay, let's see. I need to combine the numbers 12, 91, 40, and 13 using each exactly once, and reach 45. The allowed operations are addition, subtraction, multiplication, and division. Hmm. Let's start by thinking about possible combinations.\n\nFirst, since 45 is a relatively small number, maybe some subtraction or division is needed here. Let's list the numbers again: 12, 91, 40, 13. Let's think about possible pairs.\n\nWhat if I try to subtract two larger numbers to get a smaller one. For example, 91 minus 40 is 51. Then maybe combine with 13 and 12. But 51 plus 13 is 64, minus 12 is 52. Not 45. Hmm. Alternatively, 91 minus 13 is 78. Then 40 minus 12 is 28. 78 minus 28 is 50. Still not there.\n\nWait, maybe division is necessary. Let's think. For example, 40 divided by something. 40 / 10 is 4, but we don't have 10. Alternatively, 91 divided by something. Let's see. 91 divided by 13 is 7. Oh, that's interesting. Because 13 is one of the numbers. So 91 /13 =7. Then we have 12, 40, and 7 left. How can we combine those to get 45?\n\nSo 40 + 7 =47, then 47 - 12=35. No. Not 45. Alternatively, 40 -7=33. 33 +12=45. Oh! Wait, that would work. Let's check:\n\nStep 1: 91 /13 =7. Then we have 12,40,7 left. Then 40 -7=33. Then 33 +12=45. So all numbers are used exactly once: 91,13,40,12. Let's verify:\n\nFirst operation: 91 /13 =7.\n\nSecond operation: 40 -7=33. But wait, 7 is a result from the first operation, not the original numbers. So that uses 40 and 7 (which is the result of 91/13). Then third operation is 33 +12=45. So yes, all original numbers are used: 12,91,40,13. Each used exactly once. Let me check each step:\n\nOriginal numbers: 12,91,40,13.\n\nFirst step: 91 /13=7. So 91 and 13 are used. Remaining numbers:12,40, and the result 7.\n\nSecond step: 40 -7=33. Now, 40 is used. Remaining numbers:12 and 33 (from previous step).\n\nThird step: 33 +12=45. So 12 is used. All original numbers are accounted for: 91,13,40,12. Yes.\n\nSo the operations would be:\n\n['91/13=7', '40-7=33', '33+12=45'].\n\nThe final result is 45, which matches the target. So that works."}]}
"""

# flip it so when we index "original_idx" we get line['messages'][1]['reasoning_content'] + "\n" + line['messages'][1]['content']

text_only = [None] * len(lines)
for line in lines:
    idx = line['original_idx']
    text_only[idx] = (line['messages'][1]['reasoning_content'] + "\n" + line['messages'][1]['content'])
    
# remove None
text_only = [p for p in text_only if p is not None]

data = data.add_column("completion", text_only)
    
data = data.map( lambda x: {
    'messages_deepseek': [
        {'role': 'user', 'content': x['messages_sos'][0]['content']},
        {'role': 'assistant', 'content': x['completion']}
        ]
})
data = data.select_columns(["messages_deepseek", "nums", "target", "solution", "completion"])
data = data.add_column("parsed_results", list(map(evaluate_countdown_trajectory, data)))

Loading dataset from Hugging Face...


NameError: name 'evaluate_countdown_trajectory' is not defined

In [ ]:
data

Dataset({
    features: ['messages_deepseek', 'nums', 'target', 'solution', 'completion', 'parsed_results'],
    num_rows: 8000
})

In [15]:
results = sum([d['parsed_results']['solved'] for d in data]) / len(data)
print(results)
data = data.remove_columns(["completion"])

data_rejection_sampled = data.filter(lambda x: x['parsed_results']['solved'])
# data_rejection_sampled = data_rejection_sampled.rename_column("messages_deepseek", "messages")

0.7895


Filter:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [27]:
print("Loading dataset from Hugging Face...")
dataset_test = load_dataset("MelinaLaimon/stream-of-search", split="test").select(range(1000))

# Prepare test dataset
dataset_test = dataset_test.select_columns(["messages_sos", "nums", "target", "solution"])
dataset_test = dataset_test.rename_column("messages_sos", "messages_deepseek")


dataset_test = dataset_test.map( lambda x: {
    'completion': x['messages_deepseek'][1]['content'],
})


dataset_test = dataset_test.add_column("parsed_results", list(map(evaluate_countdown_trajectory, dataset_test)))
dataset_test = dataset_test.remove_columns(["completion"])

Loading dataset from Hugging Face...


NameError: name 'evaluate_countdown_trajectory' is not defined

In [28]:
from datasets import load_dataset
dataset_sos = load_dataset("MelinaLaimon/stream-of-search", split="train[0%:20%]")
dataset_sos_react = load_dataset("MelinaLaimon/stream-of-search", split="train[20%:40%]")
dataset_train_optimal = load_dataset("MelinaLaimon/stream-of-search", split="train[40%:50%]")

dataset_train_combined = {
    "sos": dataset_sos.select_columns(['messages_sos', 'nums', 'target', 'solution']),
    "sos_react": dataset_sos_react.select_columns(['messages_sos_react', 'nums', 'target', 'solution']),
    "optimal": dataset_train_optimal.select_columns(['messages_optimal', 'nums', 'target', 'solution']),
}

In [29]:
dataset_train_combined["sos"] = dataset_train_combined["sos"].rename_column("messages_sos", "messages_deepseek")
dataset_train_combined["sos_react"] = dataset_train_combined["sos_react"].rename_column("messages_sos_react", "messages_deepseek")
dataset_train_combined["optimal"] = dataset_train_combined["optimal"].rename_column("messages_optimal", "messages_deepseek")

In [30]:
dataset_deepseek = load_dataset("yeok/stream-of-search-dataset_deepseek_rejection_sampled")
dataset_deepseek['train'] = dataset_deepseek['train'].select(range(5000))

In [ ]:
from datasets import concatenate_datasets, DatasetDict
dataset_deepseek['train'] = concatenate_datasets([
    dataset_deepseek['train'],
    dataset_train_combined["sos"],
    dataset_train_combined["sos_react"],
    dataset_train_combined["optimal"],
])

In [ ]:
dataset_deepseek['train'] = dataset_deepseek['train'].shuffle(seed=42)

In [45]:
dataset_deepseek['train'][10]

{'messages_deepseek': [{'content': "Combine these initial numbers [64, 66, 11, 48] using only arithmetic operations (+, -, *, /) to reach the target value 74. All initial numbers must be used exactly once.\nConclude with the final result in EXACTLY this format:\n```\nSOLUTION: YES/NO\nOPERATIONS: list of string of operations performed, each string involving only 1 operation. For example, ['A+B=C','C+D=E'] is allowed, ['A+B+D=E'] is not allowed\nRESULT: final_value\n```\n",
   'role': 'user'},
  {'content': "Current State: 74:[64, 66, 11, 48], Operations: []\nExploring Operation: 66-64=2, Resulting Numbers: [11, 48, 2]\nGenerated Node #2: [11, 48, 2] from Operation: 66-64=2\nCurrent State: 74:[11, 48, 2], Operations: ['66-64=2']\nExploring Operation: 48-11=37, Resulting Numbers: [2, 37]\nGenerated Node #3: [2, 37] from Operation: 48-11=37\nCurrent State: 74:[2, 37], Operations: ['66-64=2', '48-11=37']\nExploring Operation: 2*37=74, Resulting Numbers: [74]\n74,74 equal: Goal Reached\n\n`

In [36]:
from datasets import concatenate_datasets, DatasetDict

new_ds = DatasetDict(dataset_deepseek)
new_ds.push_to_hub("yeok/stream-of-search-dataset_deepseek_mixed")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/cs/student/msc/ml/2024/ycheah/disk/miniconda3/envs/sos1/lib/python3.10/site-packages/huggingface_hub/lfs.py:337: UserWarning: hf_transfer is enabled but does not support uploading from bytes or BinaryIO, falling back to regular upload
  warnings.warn(


README.md:   0%|          | 0.00/822 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/yeok/stream-of-search-dataset_deepseek_mixed/commit/c916654f5eb48e09b31ff858053ea7ff97247112', commit_message='Upload dataset', commit_description='', oid='c916654f5eb48e09b31ff858053ea7ff97247112', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yeok/stream-of-search-dataset_deepseek_mixed', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yeok/stream-of-search-dataset_deepseek_mixed'), pr_revision=None, pr_num=None)

In [ ]:
from datasets import DatasetDict
dataset_total = {}
dataset_total["test"] = dataset_test
dataset_total["train"] = data_rejection_sampled
new_ds = DatasetDict(dataset_total)
new_ds.push_to_hub("yeok/stream-of-search-dataset_rejection_sampled")

from datasets import DatasetDict
dataset_total = {}
dataset_total["test"] = dataset_test
dataset_total["train"] = data
new_ds = DatasetDict(dataset_total)
new_ds.push_to_hub("yeok/stream-of-search-dataset_full")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/datasets/yeok/stream-of-search-dataset_full/commit/d44b911e1af36d5d1e25c162c385aec9c960a5b8', commit_message='Upload dataset', commit_description='', oid='d44b911e1af36d5d1e25c162c385aec9c960a5b8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yeok/stream-of-search-dataset_full', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yeok/stream-of-search-dataset_full'), pr_revision=None, pr_num=None)

In [102]:
print("Loading dataset from Hugging Face...")
dataset_test = load_dataset("MelinaLaimon/stream-of-search", split="test").select(range(1000))

dataset_total = {}

# Prepare test dataset
dataset_test = dataset_test.select_columns(["messages_sos", "nums", "target", "solution"])
dataset_test = dataset_test.rename_column("messages_sos", "messages")
dataset_total["test"] = dataset_test

Loading dataset from Hugging Face...


In [ ]:
from datasets import concatenate_datasets
data_sos = load_dataset("MelinaLaimon/stream-of-search", split="train").select(range(5000))
data_sos = data_sos.select_columns(["messages_sos", "nums", "target", "solution"])
data_sos = data_sos.rename_column("messages_sos", "messages")

dataset_total["train"] = concatenate_datasets([data_rejection_sampled.select(range(5000)), data_sos])
dataset_total["train"] = dataset_total["train"].shuffle(seed=42)


In [104]:
dataset_total["train"] = dataset_total["train"].select_columns(["messages", "nums", "target", "solution"])

In [109]:
dataset_total["train"][8000]

{'messages': [{'content': "Okay, let's see. I need to combine the numbers 33, 88, 8, and 65 using each exactly once with arithmetic operations to reach 43. Hmm. All numbers must be used, so maybe I can pair them up first and then combine the results. Let me think of different combinations.\n\nFirst, let's list the numbers: 33, 88, 8, 65. The target is 43. So maybe I can find pairs that get me closer. Let's see. For example, 88 and 8. 88 divided by 8 is 11. That's a nice number. Then I have 33 and 65 left. 33 plus 65 is 98. Then 98 minus 11 would be 87. Not 43. Alternatively, maybe 65 minus 33 is 32. 32 plus 11 is 43. Oh, wait! Wait a minute. Let's check that again.\n\nIf I do 88 / 8 = 11. Then take 65 - 33 = 32. Then add 32 + 11 = 43. That uses all four numbers. Let me verify each step. First operation: 88 divided by 8 equals 11. Then 65 minus 33 equals 32. Then 32 plus 11 equals 43. Yes, that uses all four numbers once each. So the operations would be:\n\n88 / 8 = 11 (using 88 and 8)\

In [71]:
from datasets import concatenate_datasets
data_concat = concatenate_datasets([data, data_sos])

In [93]:
dataset_total

Dataset({
    features: ['messages', 'nums', 'target', 'solution'],
    num_rows: 10000
})

In [92]:
from datasets import DatasetDict
new_ds = DatasetDict(dataset_total)
new_ds.push_to_hub("yeok/stream-of-search-dataset_deepseek_and_sos")

ValueError: dictionary update sequence element #0 has length 4; 2 is required

In [42]:
dataset_test

Dataset({
    features: ['messages', 'nums', 'target', 'solution'],
    num_rows: 1000
})

In [29]:
# rejection sample if d['parsed_results']['solved'] then remove from data
# data_rejection_sampled = data.filter(lambda x: x['parsed_results']['solved'])

data_rejection_sampled = data.filter(lambda x: x['messages_deepseek'] != "")






Filter:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [23]:
data = data.select_columns(["messages_deepseek", "nums", "target", "solution"])

In [30]:
dataset_test.select_columns(["messages_sos", "nums", "target", "solution"])

Dataset({
    features: ['messages_sos', 'nums', 'target', 'solution'],
    num_rows: 1000
})

In [32]:
data_rejection_sampled['messages_deepseek'][-5]

[{'content': "Okay, so I need to combine the numbers 72, 45, 15, and 93 using each exactly once with arithmetic operations to reach 35. Hmm. Let me think. All numbers must be used, and each operation combines two numbers into one, so with four numbers, I need three operations. Let's see.\n\nFirst, maybe I can try different groupings. Let's consider the operations step by step. Let's start by pairing some numbers. For example, if I subtract two numbers to get a smaller value. Like 72 minus 45 would be 27. Then I have 27, 15, 93 left. Hmm, not sure how to proceed from there.\n\nAlternatively, 93 minus something. Let's see. 93 minus 15 is 78. Then we have 72, 45, 78. But then 78 minus 72 is 6, and 6 minus 45 is -39. Not helpful.\n\nWait, maybe division is necessary here. Let's see. For instance, 45 divided by 15 is 3. Then we have 72, 3, 93. Then 72 divided by 3 is 24. Then 24 plus 93 is 117. No, that's too big. Not helpful.\n\nWhat about 72 divided by something? Let's think. If I take 72

In [ ]:
print("Loading dataset from Hugging Face...")
dataset_test = load_dataset("MelinaLaimon/stream-of-search", split="test").select(range(1000))

dataset_total = {}

# Prepare test dataset
dataset_test = dataset_test.select_columns(["messages_sos", "nums", "target", "solution"])
dataset_test = dataset_test.rename_column("messages_sos", "messages")
dataset_total["test"] = dataset_test

# Prepare DeepSeek dataset
dataset_total["train_deepseek"] = data_rejection_sampled.select_columns(["messages_deepseek", "nums", "target", "solution"])
dataset_total["train_deepseek"] = dataset_total["train_deepseek"].rename_column("messages_deepseek", "messages")

# Create a DatasetDict
from datasets import DatasetDict
new_ds = DatasetDict(dataset_total)


Loading dataset from Hugging Face...


In [36]:
new_ds.push_to_hub("yeok/stream-of-search-dataset_deepseek_rejection_sampled")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/cs/student/msc/ml/2024/ycheah/disk/miniconda3/envs/sos1/lib/python3.10/site-packages/huggingface_hub/lfs.py:337: UserWarning: hf_transfer is enabled but does not support uploading from bytes or BinaryIO, falling back to regular upload
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/datasets/yeok/stream-of-search-dataset_deepseek_rejection_sampled/commit/16305988035a5c4a524a11fd977dfc8dc659cb40', commit_message='Upload dataset', commit_description='', oid='16305988035a5c4a524a11fd977dfc8dc659cb40', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yeok/stream-of-search-dataset_deepseek_rejection_sampled', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yeok/stream-of-search-dataset_deepseek_rejection_sampled'), pr_revision=None, pr_num=None)

In [32]:
# only keep columns 'messages_deepseek', 'nums', 'target' 'solution
# map messages_deepseek to messages
data = data.map(lambda x: {
    'completion': x['messages_deepseek'][0]['content'],
    # 'nums': x['nums'],
    # 'target': x['target'],
    # 'solution': x['solution']
})

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from src.result_parsers.countdown_trajectories import evaluate_countdown_trajectory

data = data.add_column("parsed_results", list(map(evaluate_countdown_trajectory, data)))

results = sum([d['solved'] for d in data['parsed_results']]) / len(data['parsed_results'])

In [1]:
from datasets import load_dataset

dataset = load_dataset("yeok/stream-of-search-dataset_deepseek_rejection_sampled")

In [ ]:
# rename split train_deepseek into train
ds = dict(dataset)
ds['train'] = ds['train_deepseek'] 
del ds['train_deepseek']

new_ds.push_to_hub("yeok/stream-of-search-dataset_deepseek_rejection_sampled_5k")

In [ ]:
new_ds.push_to_hub("yeok/stream-of-search-dataset_deepseek_rejection_sampled_5k")

{'test': Dataset({
     features: ['messages', 'nums', 'target', 'solution'],
     num_rows: 1000
 }),
 'train': Dataset({
     features: ['messages', 'nums', 'target', 'solution'],
     num_rows: 6316
 })}